In [1]:
import spacy
nlp = spacy.load('en_core_web_lg')

import numpy as np
import pandas as pd

In [2]:


df1 = pd.read_csv('Cleaned-P&P data.csv')

blanks = []  # start with an empty list`

for i,c,d,inc,sd in df1.itertuples():  # iterate over the DataFrame
        if c == 'O' or c=='E':         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
df1.drop(blanks, inplace=True)
df1["desc"] = df1["short_des"] + '. ' + df1["desc"]
df1.reset_index(drop=True,inplace=True)



df2 = pd.read_csv('Cleaned-P&P data.csv')
blanks2 = []  # start with an empty list`

for i,c,d,inc,sd in df2.itertuples():  # iterate over the DataFrame
        if c == 'O' or c=='E':         # test 'review' for whitespace
            blanks2.append(i)     # add matching index numbers to the list
df2.drop(blanks2, inplace=True)
df2.reset_index(drop=True,inplace=True)

dataset = pd.concat([df1, df2], ignore_index=True)

# Cleaning the texts
import re
corpus = []
all_words = []
max_len=0
for i in range(0, 490):

    review = re.sub('\w\d{7}', ' ', dataset['desc'][i])
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = ' '.join(review)
    
    review = nlp(review)
    review = [word.text for word in review]
   # for word.text in review:
        
    all_words.append(review) 
    if len(review) > max_len:
        max_len = len(review)
    ds = ' '.join(review)
#    for word2vec we want an array of vectors
    corpus.append(ds)
    

In [3]:
len(dataset)

490

### Standardise key terms in data

In [4]:
dpf = pd.read_csv('Del-Pass.csv')



dpf['kpattern']

keys_list = dpf['kpattern'].tolist()
regex = re.compile(r'(' + '|'.join(keys_list) + r')') 

cus_list = ['remove','removed','cancelled', 'cancel' ,'deleted']
regey = re.compile(r'(' + '|'.join(cus_list) + r')') 

import nltk
from nltk.corpus import stopwords
stopwords=nltk.corpus.stopwords.words('english')
additional =["thank you","please","hello","hi","advise","hin't","st","nd","rd","th","thank","pnt","work request","good morning","good afternoon","short description","jd williams","nbrown"]
stopwords = stopwords + additional
remov=[]
for ele in stopwords:
    matches = re.findall("n't",ele)
    matches2 = re.findall("'nt",ele)
    if len(matches)>0 or len(matches2)>0:
        remov.append(ele)
stopwords = [word for word in stopwords if word not in remov]




In [5]:
cldict = {'P':1 , 'R':1, 'O':-1,'E':-1}
dataset['Class Label'] = dataset['class'].map(cldict)
label_series = pd.Series(dataset['Class Label'])

from keras.utils import to_categorical

class_label = np.array(label_series)
y = class_label
y.shape

X = corpus



Using TensorFlow backend.


In [6]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=47)

### Create custom rules

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
class SpacyVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp
        self.dim = 300
        pass
    def fit(self, X, y):
        return self

    def transform(self, X):
        # Doc.vector defaults to an average of the token vectors.
        # https://spacy.io/api/doc#vector
        lemmatized_list=[]
        for snt in X:
        #    print(snt)
            tokens = nlp(snt)
        #    for token in tokenized:
            filtered_sentence = [w.lemma_ for w in tokens if not w.text in stopwords]
        #                 lemm = token.lemma_ for token.text in token
        #             lemmatized_list.append(lemm)
            stri = ' '.join(filtered_sentence)
            lemmatized_list.append(stri)
        return [self.nlp(text).vector for text in lemmatized_list]

In [8]:
class Lemm(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass   
  #  def fit(self, X, y=None):
    def fit(self, X, y):
        return self    
    def transform(self, X):       
        lemmatized_list=[]
        for snt in X:
        #    print(snt)
            tokens = nlp(snt)
        #    for token in tokenized:
            filtered_sentence = [w.lemma_ for w in tokens if not w.text in stopwords]
        #                 lemm = token.lemma_ for token.text in token
        #             lemmatized_list.append(lemm)
            stri = ' '.join(filtered_sentence)
            lemmatized_list.append(stri)
        return lemmatized_list

In [9]:
#segment_featurizer = SegmentFeaturizer()  # more on this below
class CustomLinguisticFeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass   
  #  def fit(self, X, y=None):
    def fit(self, X, y):
        return self    
    def transform(self, X):       
        ref_corpus=[]
        for text in X:           
            trans = regex.sub(lambda m: m.group().replace(m.group(),"delpass "),text)
            trans_ref = regey.sub(lambda n: n.group().replace(n.group(),"delete "),trans)
             
            ref_corpus.append(trans_ref)   
        return ref_corpus


In [10]:

class CusttransformFeat(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass   
  #  def fit(self, X, y=None):
    def fit(self, X, y):
        return self    
    def transform(self, X):       
        X = np.array(X)
        #vectorizer = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("train 1 class tfidf vocab.pkl", "rb")))
        vectorizer = TfidfVectorizer(vocabulary=pickle.load(open("BigVocabTfidf 1 class.pkl", "rb")))
        vec_train = vectorizer.fit_transform(X).toarray()
        #pickle.dump(vectorizer.vocabulary_,open("BigVocabTfidf 1 class.pkl","wb"))
        #print(vec_train)
        return vec_train
        

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer  
import pickle
from sklearn.preprocessing import Normalizer
#from sklearn.feature_extraction.text import HashingVectorizer
#vectorizer = HashingVectorizer(n_features=20)
embeddings_pipeline4 = Pipeline(
    steps=[
        ("cust",CustomLinguisticFeatureTransformer()),
        ("lemm", Lemm()),
        
#        ("Hash", HashingVectorizer(n_features=644)),
        ("tftra", CusttransformFeat()),
#        ("mean_embeddings", SpacyVectorTransformer(nlp)),
#         ("reduce_dim", TruncatedSVD(200)),
#         ("dict_vect", DictVectorizer()),
        ("classifier", OneClassSVM(gamma=0.1,kernel='rbf', nu=0.1)),
    ]
)
embeddings_pipeline4.fit(X, y)

Pipeline(memory=None,
     steps=[('cust', CustomLinguisticFeatureTransformer()), ('lemm', Lemm()), ('tftra', CusttransformFeat()), ('classifier', OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=False))])

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
lst1=['delivery pass purchased on staff accounts needs removing please attached spreadsheet with the updated list of accounts thank you']
transformer = TfidfTransformer()
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("train 1 class tfidf vocab.pkl", "rb")))
tfidf = transformer.fit_transform(loaded_vec.fit_transform(np.array(lst1))).toarray()

In [13]:
from sklearn.metrics import classification_report
y_pred = embeddings_pipeline4.predict(X)
cr = classification_report(y, y_pred)
print(cr)

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           1       1.00      0.89      0.94       490

   micro avg       0.89      0.89      0.89       490
   macro avg       0.50      0.44      0.47       490
weighted avg       1.00      0.89      0.94       490



C:\Users\AVNEET\miniconda3\envs\nlp_course\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [14]:

from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y,y_pred)

array([[  0,   0],
       [ 54, 436]], dtype=int64)

In [24]:
accuracy_score(y,y_pred)

0.889795918367347

In [15]:
lst1=['this customer is being charged for delivery after purchasing the subscription please can you see if it needs activating']
lst2 = np.array(lst1)
# lst3 = np.append(trainx, lst2)
y_pr = embeddings_pipeline4.predict(lst1)
y_pr

array([1], dtype=int64)

In [16]:
ind = []
for i in range(0,392):
    if y_train[i] == -1:
  #      y_train.pop[i]
         ind.append(i)
X_tran = np.array(X_train)
trainy = np.delete(y_train, ind)
trainx = np.delete(X_tran, ind)

trainx=list(trainx)

In [17]:
for i in range(0,490):
    if y[i]!=y_pred[i]:
        if(y_pred[i] == -1):
            print(X[i],'***',y_pred[i])

good afternoon this customer is being charged for delivery after purchasing the subscription please can you see if it needs activating thank you in advance rebecca connolly proactive notification team *** -1
good afternoon i have a query from a customer regarding their delivery subscription as they are still being charged for delivery please can you check as to whether it needs activating thank you in advance rebecca connolly proactive notification team *** -1
customer charged for next day delivery ashley reported that the customer has a delivery subscription purchased for which she has free delivery on the next day deliveries for the orders but while placing orders she is still charged for next day delivery or even the standard delivery please check orderno customerno website fashion world contact *** -1
hi the above customer can not add the delivery subscription to her account i have looked at the account today to remove any delivery subscription and currently there is none previousl

In [19]:
pickle.dump(embeddings_pipeline4,open("OneClassTFIDF_V_0.2.pkl","wb"))

In [ ]:
for i in range(0,490):
    if y[i]== -1:
        print(X[i],'****', y[i])

In [20]:
## load model


loaded_model = pickle.load(open('OneClassTFIDF_V_0.2.pkl','rb'))

In [21]:
lst1=['q35 job pwani3md rc0012  step02 pwani3md rc0012  step02  restart  rerun  new instructions 08052020 until further notice  purge the rerun and raise an inc assigned  to vineet agarwal e pss']

y_pr = loaded_model.predict(lst1)
#crpt = classification_report(y, y_pred)
y_pr

array([-1], dtype=int64)

In [22]:
from joblib import dump, load
dump(embeddings_pipeline4, filename = 'OneClassTFIDF_V_0.3.joblib' )

['OneClassTFIDF_V_0.3.joblib']

In [ ]:
print(crpt)

In [ ]:
confusion_matrix(y,y_pred)